- you've seen Kalman filters in the context of localization, but they're much more general
- you can apply them to almost any tracking problem in physical systems
- we're going to talk about sensor fusion, and apply Kalman filters in their contexts, and to throw one more wrench, you're going to do it in C++

- in this lesson, you will learn how to develop a fusion system by using Kalman filters
- after the lesson with Sebastian, you should already have a good sense of how to generally apply the Kalman filter to estimation problems
  - you've only used it to combine sequential measurements from one sensor
  - but actually, it's also possible to use Kalman filters to combine measurements from different sensors, and that's why we use it in sensor fusion


- the plan for the project in this module is to use sensor fusion to track a pedestrian
- you have already learned that lidar and  radar have their individual strengths and weaknesses
- by combining the strengths of both of these sensors, you'll be able to estimate precisely the pedestrian location, heading, and speed

# Lesson Map and Fusion Flow

- so far you've learned a lot about car sensors, and also about how to implement a Kalman filter in Python
- we also expect that you've got enough of a grasp on C++ to the point where you can now implement a Kalman filter in it
- in this lesson you're going to combine all of this knowledge to develop a complete fusion model
- there are many pieces in a Kalman filter based sensor fusion, so I want to quickly show you a general map of what you'll be building


- first off, you're going to build something called an Extended Kalman Filter
  - it's extended in the sense that it will be capable of handling more complex motion models and measurement models
- you have two sensors, a lidar and a radar
- the information provided by these sensors is used to estimate the state of a moving pedestrian
  - this state is represented by a 2D position and a 2D velocity
- each time we receive new measurements from a given sensor, the estimation function is triggered
  - at this point we perform two steps, State Prediction and Measurement Update
  - in the prediction step, we predict the pedestrian state and its covariance
    - we do so by taking into account the elapsed time between the current and the previous observations
  - the measurement update step depends on sensor type
    - there are two cases that we'll see, Laser and Radar
    - if the current measurements are generated by a laser sensor, then we just apply a standard Kalman filter to update the pedestrian's state
    - however, radar measurements involve a nonlinear measurement function
      - so when we receive radar measurements, we use different tweaks to handle the measurement update
      - for instance, we may use Extended Kalman filter equations, which you'll see later


- here is the overall processing flow:

<img src="resources/kalman_filter_algorithm_map.png"/>

- imagine you are in a car equipped with sensors on the outside
- the car sensors can detect objects moving around: for example, the sensors might detect a pedestrian, as described in the video, or even a bicycle
- for variety, let's step through the Kalman Filter algorithm using the bicycle example


- the Kalman Filter algorithm will go through the following steps:
  - **first measurement**
    - the filter will receive initial measurements of the bicycle's position relative to the car
    - these measurements will come from a radar or lidar sensor
  - **initialize state and covariance matrices**
    - the filter will initialize the bicycle's position based on the first measurement
  - then the car will receive another sensor measurement after a time period $\Delta{t}$
  - **predict**
    - the algorithm will predict where the bicycle will be after time $\Delta{t}$
    - one basic way to predict the bicycle location after $\Delta{t}$ is to assume the bicycle's velocity is constant; thus the bicycle will have moved velocity * $\Delta{t}$
    - in the extended Kalman filter lesson, we will assume the velocity is constant
  - **update**
    - the filter compares the "predicted" location with what the sensor measurement says
    - the predicted location and the measured location are combined to give an updated location
    - the Kalman filter will put more weight on either the predicted location or the measured location depending on the uncertainty of each value
  - then the car will receive another sensor measurement after a time period $\Delta{t}$
    - the algorithm then does another **predict** and **update** step

# Estimation Problem Refresh

- let's go back to the problem
- we want to track a pedestrian moving in front of our autonomous vehicle

<img src="resources/track_pedestrians_problem.png"/>

- at its core, the Kalman filter is a two-step estimation problem, predict and then update
  - we always start with what we already know from the past about the pedestrian
  - we use that information to predict the state of the pedestrian up to the time when the next measurement arrives, and this is called the prediction step
  - the next step is called the update step, where we essentially use the new observations to correct our belief about the state of the pedestrian
- the Kalman filter simply consists of an endless loop of prediction/update steps; this is what you built with Sebastian earlier in this module


- but what happens when there are two sensors that observe the same pedestrian?
- how does that change the Kalman filter?
- actually, we can keep the same processing flow with the difference that each sensor is going to have its own prediction/update scheme

<img src="resources/two_sensors_pedestrian_observation.png"/>

- in other words, the belief about the pedestrian's position and velocity is updated asynchronously each time new measurement is received regardless the source sensor
- let's step through an example flow to see how it works

<img src="resources/estimation_problem_example_flow.png"/>

- here the pedestrian state at time $k$ is a distribution with its mean $x$ and covariance $P$
- let's say we're now at time $k + 1$, and we've just received the laser measurement
  - the first thing we do before we look at the measurement update is to make a prediction about where we think the pedestrian from time $k$ will be at time $k + 1$
  - the second thing we do is the so called measurement update, where we combine the pedestrian's predicted state with the new laser measurement
  - what we now have is a more accurate belief about the pedestrian's position at time $k + 1$; this is what we call the posterior
- now let's imagine that we receive the radar measurement at time $k + 2$
  - first, we again predict the pedestrian state from $k + 1$ to $k + 2$
    - note, this prediction for radar is exactly the same function as in the laser case
  - what changes, in this case, is the Measurement Update step
    - as we know, the radar sees the word differently than laser
    - while laser provides measurement in a Cartesian coordinate system, radar provides measurement in a polar coordinate system
    - thus we have to use different measurement update functions specific to radar data, so this is a more detailed view of the Kalman filter
- we received the measurements from different sensors at each timestamp, and then we make a prediction followed by a measurement update

### Definition of Variables

  - $x$ is the mean state vector
    - for an extended Kalman filter, the mean state vector contains information about the object's position and velocity that you are tracking
    - it is called the "mean" state vector because position and velocity are represented by a gaussian distribution with mean $x$
  - $P$ is the state covariance matrix, which contains information about the uncertainty of the object's position and velocity
    - you can think of it as containing standard deviations
  - $k$ represents time steps
    - so $x_k$ refers to the object's position and velocity vector at time $k$
  - the notation $k + 1∣k$ refers to the prediction step
    - at time $k + 1$, you receive a sensor measurement
    - before taking into account the sensor measurement to update your belief about the object's position and velocity, you predict where you think the object will be at time $k+1$
    - you can predict the position of the object at $k + 1$ based on its position and velocity at time $k$
    - hence $x_{k + 1 ∣ k}$ means that you have predicted where the object will be at $k+1$ but have not yet taken the sensor measurement into account
  - $x_{k + 1}$ means that you have now predicted where the object will be at time $k + 1$ and then used the sensor measurement to update the object's position and velocity

- what should a Kalman Filter do if both the radar and laser measurements arrive at the same time, $k + 3$?
  - hint: The Kalman filter algorithm predicts -> updates -> predicts -> updates, etc.
    - if two sensor measurements come in simultaneously, the time step between the first measurement and the second measurement would be zero


- it would predict the state to $k + 3$ then use either one of the sensors to update
- then predict the state to $k + 3$ again and update with the other sensor measurement
- as you saw, the Kalman filter is a two-step process: predict, and then update
- if you receive two measurements simultaneously, you can use this process with either measurement and then repeat the process with the other measurement
  - the order does not matter!

<img src="resources/two_sensors_same_time_measure_update_quiz.png"/>

- because we have already run a prediction-update iteration with the first sensor at time $k + 3$, the output of the second prediction at time $k + 3$ will actually be identical to the output from the update step with the first sensor
- so, in theory, you could skip the second prediction step and just run a prediction, update, update iteration

# Kalman Filter Intuition


- let me give you a quick reminder of the Kalman filter for a simple 1D motion case
- let's say that our mission is to track a pedestrian whose state, $x$, is described by a position and velocity
- when designing the Kalman filter, we have to define the two linear functions
  - the state transition function, that models how the state is changed from time $k - 1$ to time $k$
  - the measurement function, that models how the measurement is  calculated, and how it's related to the predicted state $x$
- the first terms in these functions represent the deterministic part of our model
- the last terms, $\nu$ noise and $\omega$ noise, represent the stochastic part
  - in other words, random noises that affect the prediction and measurement update steps

<img src="resources/kalman_linear_functions_transition_and_measurement.png"/>

- by using the linear motion model with a constant velocity, the new location, $p$, of our pedestrian is the old location + velocity times $\Delta t$
- because velocity is constant,  the new velocity is the same as the old velocity
  - we can express this in a matrix form like so
- for the measurement function, our vehicle only senses the pedestrian position, so the measurement function looks like this
  - it also can be represented in a matrix form as so

<img src="resources/linear_motion_model_measurement_function.png"/>

- as we already know, the Kalman filter algorithm is composed of a prediction step where we predict our state and covariance $p$
- we also use a measurement update step, often called correction step, where we use the latest measurement to update the state estimate and its uncertainty

<img src="resources/prediction_and_measurement_update_steps.png"/>

- the Kalman equation contains many variables, so here is a high level overview to get some intuition about what the Kalman filter is doing

### Prediction

- let's say we know an object's current position and velocity, which we keep in the $x$ variable
- now one second has passed
- we can predict where the object will be one second later because we knew the object position and velocity one second ago; we'll just assume the object kept going at the same velocity
- the $x' = Fx + \nu$ equation does these prediction calculations for us


- but maybe the object didn't maintain the exact same velocity
- maybe the object changed direction, accelerated or decelerated
- so when we predict the position one second later, our uncertainty increases
- $P'= FPF^T + Q$ represents this increase in uncertainty


- process noise refers to the uncertainty in the prediction step
- we assume the object travels at a constant velocity, but in reality, the object might accelerate or decelerate
- the notation $\nu \sim N(0,Q)$ defines the process noise as a gaussian distribution with mean zero and covariance $Q$


### Update

- now we get some sensor information that tells where the object is relative to the car
- first we compare where we think we are with what the sensor data tells us $y = z − Hx'$


- the $K$ matrix, often called the Kalman filter gain, combines the uncertainty of where we think we are $P'$ with the uncertainty of our sensor measurement $R$
- if our sensor measurements are very uncertain ($R$ is high relative to $P'$), then the Kalman filter will give more weight to where we think we are: $x'$
- if where we think we are is uncertain ($P'$ is high relative to $R$), the Kalman filter will put more weight on the sensor measurement: $z$


- measurement noise refers to uncertainty in sensor measurements
- the notation $\omega \sim N(0, R)$ defines the measurement noise as a gaussian distribution with mean zero and covariance $R$
- measurement noise comes from uncertainty in sensor measurements

### A Note About the State Transition Function: Bu

- if you go back to the video, you'll notice that the state transition function was first given as $x' = Fx + Bu + \nu$
- but then $Bu$ was crossed out leaving $x' = Fx + \nu$
- $B$ is a matrix called the control input matrix and $u$ is the control vector


- as an example, let's say we were tracking a car and we knew for certain how much the car's motor was going to accelerate or decelerate over time; in other words, we had an equation to model the exact amount of acceleration at any given moment
- $Bu$ would represent the updated position of the car due to the internal force of the motor
- we would use $\nu$ to represent any random noise that we could not precisely predict like if the car slipped on the road or a strong wind moved the car


- for the Kalman filter lessons, we will assume that there is no way to measure or know the exact acceleration of a tracked object
- for example, if we were in an autonomous vehicle tracking a bicycle, pedestrian or another car, we would not be able to model the internal forces of the other object; hence, we do not know for certain what the other object's acceleration is
- instead, we will set $Bu = 0$ and represent acceleration as a random noise with mean $\nu$

# Kalman Filter Equations in C++

- now, let's do a quick refresher of the Kalman Filter for a simple 1D motion case
- let's say that your goal is to track a pedestrian with state $x$ that is described by a position and velocity
  - $x = \begin{pmatrix} p \\ v \end{pmatrix}$

### Prediction Step

- when designing the Kalman filter, we have to define the two linear functions: the state transition function and the measurement function
- the state transition function is $x' = F * x + noise$, where $F = \begin{pmatrix} 1 & \Delta t \\ 0 & 1 \end{pmatrix}$ and $x'$ is where we predict the object to be after time $\Delta t$
  - $F$ is a matrix that, when multiplied with $x$, predicts where the object will be after time $\Delta t$
- by using the linear motion model with a constant velocity, the new location, $p'$ is calculated as $p' = p + v * \Delta t$, where $p$ is the old location and $v$, the velocity, will be the same as the new velocity ($v' = v$) because the velocity is constant
  - we can express this in a matrix form as follows: $\begin{pmatrix} p' \\ v' \end{pmatrix} = \begin{pmatrix}1 & \Delta t \\ 0 & 1 \end{pmatrix} \begin{pmatrix} p \\ v \end{pmatrix}$


- remember we are representing the object location and velocity as Gaussian distributions with mean $x$
- when working with the equation $x' = F * x + noise$, we are calculating the mean value of the state vector
- the noise is also represented by a gaussian distribution but with mean zero; hence, noise = 0 is saying that the mean noise is zero
  - the equation then becomes $x' = F * x$
- but the noise does have uncertainty
  - the uncertainty shows up in the $Q$ matrix as acceleration noise

### Update Step

- for the update step, we use the measurement function to map the state vector into the measurement space of the sensor
- to give a concrete example, lidar only measures an object's position but the extended Kalman filter models an object's position and velocity
- so multiplying by the measurement function $H$ matrix will drop the velocity information from the state vector $x$
- then the lidar measurement position and our belief about the object's position can be compared


- $z = H * x + w$ where $w$ represents sensor measurement noise
- so for lidar, the measurement function looks like this: $z = p'$
  - it also can be represented in a matrix form: $z = \begin{pmatrix} 1 & 0 \end{pmatrix} \begin{pmatrix} p' \\ v' \end{pmatrix}$
  
 
- as we already know, the general algorithm is composed of a prediction step where I predict the new state and covariance, $P$
- we also have a measurement update (or also called many times correction step) where we use the latest measurements to update our estimate and our uncertainty

## Implement the C++ program for the Kalman filter update equations

- for matrix manipulation, I would like you to use the Eigen library; it has no dependencies and it's really easy to use
  - in order to perform matrix operations, I have to include the Eigen/Dense header


- here is a download link to the [Eigen Library](https://d17h27t6h515a5.cloudfront.net/topher/2017/March/58b7604e_eigen/eigen.zip) that is being used throughout the programming assignments
- further details regarding this library can be found [here](http://eigen.tuxfamily.org/)


### Notes for using the Eigen Library:

- you can create a vertical vector of two elements with a command like this: 

```cpp
VectorXd my_vector(2);
```

- if the vector size was previously specified, you can use the so-called comma initializer to set all the coefficients to some values:

```cpp
my_vector << 10, 20;
```

- you can use the `cout` command to print out the vector:

```cpp
cout << my_vector << endl;
```

- the matrices can be created in the same way
  - for example, this is an initialization of a 2 by 2 matrix with the values 1, 2, 3, and 4:

```cpp
MatrixXd my_matrix(2,2);
my_matrix << 1, 2,
             3, 4;
```

- you can use the same comma initializer or you can set each matrix value explicitly
  - for example, that's how we can change the matrix elements in the second row:

```cpp
my_matrix(1,0) = 11;    //second row, first column
my_matrix(1,1) = 12;    //second row, second column
```

- you can compute the transpose of a matrix with the following command:

```cpp
MatrixXd my_matrix_t = my_matrix.transpose();
```

- you can get the matrix inverse with:

```cpp
MatrixXd my_matrix_i = my_matrix.inverse();
```

- for multiplying the matrix *m* with the vector *b* you can write this in one line as let’s say matrix *c* equals *m* times *v*:

```cpp
MatrixXd another_matrix;
another_matrix = my_matrix*my_vector;
```

- note that in the quiz below, in the `filter()` function, we actually do the measurement and then the prediction in the loop
- over time, the order of these doesn't have a huge impact, since it is just a cycle from one to the other
- here, the first thing you need is a measurement because otherwise there is no location information or even information that the object exists unless a sensor picked it up
- so, you initialize location values with the measurement

- code is available in `code/01_kalman_filter_equations_in_cpp/`

- you've just implemented the C++ Kalman filter for one dimensional tracking problem


**Q:** Why do we not use the process noise in the state prediction function, even though the state transition equation has one? In other words, why does the code set `u << 0, 0` for the equation $x = F * x + u$?


**A:** Because the noise mean is zero. Looking closely at the process noise, we know from the Kalman Filter algorithm that its mean is zero and its covariance matrix is usually noted by $Q∗N(0,Q)$ The first equation only predicts the mean state. As the mean value of the noise is zero, it does not directly affect the predicted state. However, we can see that the noise covariance $Q$ is added here to the state covariance prediction so that the state uncertainty always increases through the process noise.

# State Prediction

- now let's say that our vehicle sensors are able to provide the measurements in 2D
- at this point we need to rethink the predictions from the lessons map
  - first, we have define the state transition matrix, $F$, and the process covariance matrix, $Q$


- first, as our pedestrian is moving along both horizontal and vertical directions
- we now want to estimate a 2D position and 2D velocity
  - so we have a 4D state vector $x = \begin{pmatrix} p_x \\ p_y \\ v_x \\ v_y \\ \end{pmatrix}$
  

- second, we'll use the same linear motion model with the custom velocity
  - so, the new $x$ and $y$ position will be the old positions + the displacement, which is the same as the velocity * delta t
    - $p_x' = p_x + v_x\Delta t + \nu_{px}$
    - $p_y' = p_y + v_y\Delta t + \nu_{py}$
  - the velocity along both $x$ and $y$ axes stays the same  
    - $v_x' = v_x + \nu_{vx}$
    - $v_y' = v_y + \nu_{vy}$
  - these are kinematic formulas


- this gives us the following state transition equation: $\begin{pmatrix} p_x' \\ p_y' \\ v_x' \\ v_y' \\ \end{pmatrix} = \begin{pmatrix} 1 & 0 & \Delta t & 0 \\\ 0 & 1 & 0 & \Delta t \\\ 0 & 0 & 1 & 0 \\\ 0 & 0 & 0 & 1 \end{pmatrix} \begin{pmatrix} p_x \\ p_y \\ v_x \\ v_y \end{pmatrix} + \begin{pmatrix} \nu_{px} \\ \nu_{py} \\ \nu_{vx} \\ \nu_{vy} \end{pmatrix}$
- as a reminder, the above equation is $x' = Fx + noise$
  

- the third main change is that $\Delta t$ is not constant anymore
  - in reality, the time elapsed between two consecutive observations might vary

- but let's look a bit closer at the **motion noise** $\nu$, because you didn't really use this before with Sebastian


- motion noise and process noise refer to the same case: uncertainty in the object's position when predicting location
- the model assumes velocity is constant between time intervals, but in reality we know that an object's velocity can change due to acceleration
- the model includes this uncertainty via the process noise


- measurement noise refers to uncertainty in sensor measurements, which will be discussed in more detail later

**Q:** Suppose you have a pedestrian state $X$. I want you to compare two scenarios: in the first predict the state $0.1s$ into the future and in the second $5s$ into the future. Which of these two scenarios leads to a higher uncertainty? In answering this, consider whether or not random noise has an increasing effect with increasing gaps between prediction times.

**A:** A time difference of $5s$ leads to a higher uncertainty. Here's another way of thinking about it: if you split the $5s$ time difference into several intermediate predictions with a difference of $0.1s$, then compared to the first case, you will predict the same state many more times without receiving any feedback about the object's new position. Thus, the uncertainty increases.

**Q:** Let's say we use our linear motion model with fixed time increments, but the pedestrian is randomly changing her velocity (accelerating), sometimes speeding up, slowing down or changing direction. However, the overall mean change is zero. This introduces a noise in the tracking process - what kind of noise is it?

**A:** It's process noise. The prediction equation treats the pedestrian's velocity as constant. As such, a randomly accelerating pedestrian creates a process noise.

- from the examples I’ve just showed you we can clearly see that the process noise depends on both: the elapsed time and the uncertainty of acceleration

# Process Covariance Matrix

## Calculating Acceleration Noise Parameters

- before we discuss the derivation of the process covariance matrix $Q$, you might be curious about how to choose values for $\sigma_{ax}^2$ and $\sigma_{ay}^2$
- for the extended Kalman filter project, you will be given values for both

## Process Covariance Matrix Q - Intuition

- as a reminder, here are the state covariance matrix update equation and the equation for $Q$
- $P' = FPF^T + Q$


- $Q = \begin{pmatrix} \frac{\Delta t^4}{{4}}\sigma_{ax}^2 & 0 & \frac{\Delta t^3}{{2}}\sigma_{ax}^2 & 0 \\ 0 & \frac{\Delta t^4}{{4}}\sigma_{ay}^2 & 0 & \frac{\Delta t^3}{{2}}\sigma_{ay}^2 \\ \frac{\Delta t^3}{{2}}\sigma_{ax}^2 & 0 & \Delta t^2\sigma_{ax}^2 & 0 \\ 0 & \frac{\Delta t^3}{{2}}\sigma_{ay}^2 & 0 & \Delta t^2\sigma_{ay}^2 \end{pmatrix}$


- because our state vector only tracks position and velocity, we are modeling acceleration as a random noise
- the $Q$ matrix includes time $\Delta t$ to account for the fact that as more time passes, we become more uncertain about our position and velocity
  - so as $\Delta t$ increases, we add more uncertainty to the state covariance matrix $P$

- we need the process covariance matrix to model the stochastic part of the state transition function
- first, I'm going to show you how the acceleration is expressed by the kinematic equations
- then, I'm going to use that information to derive the process covariance matrix, $Q$


- say we have two consecutive observations of the same pedestrian with initial and final velocities
- from the kinematic formulas, we can derive the current position and speed as a function of previous state variables, including the change in the velocity, or in other words, including the acceleration $a = \dfrac{\Delta v}{\Delta t} = \dfrac{v_{k+1} - v_k}{\Delta t}$


- combining both 2D position and 2D velocity equations previously deducted formulas we have:
$\begin{cases} p_x' = p_x + v_x \Delta t + \frac{a_x \Delta t^2}{{2}}\\ p_y' = p_y + v_y \Delta t + \frac{a_y \Delta t^2}{{2}}\\ v_x' = v_x + a_x \Delta t\\ v_y' = v_y + a_y \Delta t \end{cases}$
  - looking at the deterministic part of our motion model, we assume that the velocity is constant
    - however, in reality, the pedestrian speed might change

- since the acceleration is unknown, we can add it to the noise component
    - this random noise would be expressed analytically as in the last terms in the equation
- so, we have a random acceleration vector $\nu$ in this form: $\nu = \begin{pmatrix} \nu_{px} \\ \nu_{py} \\ \nu_{vx} \\ \nu_{vy} \end{pmatrix} = \begin{pmatrix} \frac{a_x \Delta t^2}{{2}} \\ \frac{a_y \Delta t^2}{{2}} \\ a_x \Delta t \\ a_y \Delta t \end{pmatrix}$ which is described by a zero mean and a covariance matrix $Q$, so $\nu \sim N(0,Q)$


- this vector $\nu$ can be decomposed into two components: a 4 by 2 matrix $G$ which does not contain random variables and a 2 by 1 matrix $a$ which contains the random acceleration components: $\nu = \begin{pmatrix} \frac{a_x \Delta t^2}{{2}} \\ \frac{a_y \Delta t^2}{{2}} \\ a_x \Delta t \\ a_y \Delta t \end{pmatrix} = \begin{pmatrix} \frac{\Delta t^2}{{2}} & 0 \\ 0 & \frac{\Delta t^2}{{2}} \\ \Delta t & 0 \\ 0 & \Delta t \end{pmatrix} \begin{pmatrix} a_x\\ a_y \end{pmatrix} = Ga$


- $\Delta t$ is computed at each Kalman filter step and the acceleration is a random vector with zero mean and standard deviations, $\sigma_{ax}^2$ and $\sigma_{ay}^2$
  - $a_x \sim N(0,\sigma_{ax}^2)$
  - $a_y \sim N(0,\sigma_{ay}^2)$

- based on our noise vector we can define now the new covariance matrix $Q$
- the covariance matrix is defined as the expectation value of the noise vector $\nu$ times the noise vector $\nu^T$
  - $Q = E[\nu \nu^T] = E[Gaa^TG^T]$

- as matrix $G$ does not contain random variables, we can put it outside the expectation calculation
  - $Q = G E[aa^T] G^T = G \begin{pmatrix} \sigma_{ax}^2 & \sigma_{axy} \\ \sigma_{axy} & \sigma_{ay}^2 \end{pmatrix} G^T = G Q_{\nu} G^T$
- this leaves us with three statistical moments:
  - the expectation of ax times ax, which is the variance of ax squared: $\sigma_{ax}^2$
  - the expectation of ay times ay, which is the variance of ay squared: $\sigma_{ay}^2$
  - the expectation of ax times ay, which is the covariance of ax and ay: $\sigma_{axy}$


- $a_x$ and $a_y$ are assumed uncorrelated noise processes
  - this means that the covariance $\sigma_{axy}$ in $Q_{\nu}$ is zero: $Q_{\nu} = \begin{pmatrix} \sigma_{ax}^2 & \sigma_{axy} \\ \sigma_{axy} & \sigma_{ay}^2 \end{pmatrix} = \begin{pmatrix} \sigma_{ax}^2 & 0 \\ 0 & \sigma_{ay}^2 \end{pmatrix}$


- so after combining everything in one matrix we obtain our 4 by 4 $Q$ matrix: $Q = G Q_{\nu} G^T = \begin{pmatrix} \frac{\Delta t^4}{{4}}\sigma_{ax}^2 & 0 & \frac{\Delta t^3}{{2}}\sigma_{ax}^2 & 0 \\ 0 & \frac{\Delta t^4}{{4}}\sigma_{ay}^2 & 0 & \frac{\Delta t^3}{{2}}\sigma_{ay}^2 \\ \frac{\Delta t^3}{{2}}\sigma_{ax}^2 & 0 & \Delta t^2\sigma_{ax}^2 & 0 \\ 0 & \frac{\Delta t^3}{{2}}\sigma_{ay}^2 & 0 & \Delta t^2\sigma_{ay}^2 \end{pmatrix}$


- note on notation
  - some authors describe $Q$ as the complete process noise covariance matrix
  - and some authors describe $Q$ as the covariance matrix of the individual noise processes
  - in our case, the covariance matrix of the individual noise processes matrix is called $Q_\nu$, which is something to be aware of

# Laser Measurements

- so far, we have defined the motion model
- at this point, let's say a Lidar measurement has come in
- let's design our measurement model for a laser sensor in 2D
  - this will involve defining the right measurement vector $z$, the measurement matrix $H$, and the covariance matrix $R$


- the Lidar sensor output is a point cloud, but for simplicity we consider here that we already analyzed  the point cloud to compute the 2D location of the pedestrian
  - so our measurement vector is just: $z = \begin{pmatrix} p_x \\ p_y \end{pmatrix}$


- we also know that our state is a 4D vector $x = \begin{pmatrix} p_x \\ p_y \\ v_x \\ v_y \end{pmatrix}$

## Variable Definitions

- to reinforce was what discussed in the video, here is an explanation of what each variable represents:


- $z$ is the measurement vector
  - for a lidar sensor, the $z$ vector contains the $position_x$ and $position_y$ measurements


- $H$ is the matrix that projects your belief about the object's current state into the measurement space of the sensor
  - for lidar, this is a fancy way of saying that we discard velocity information from the state variable since the lidar sensor only measures position
    - the state vector $x$ contains information about $[p_x, p_y, v_x, v_y]$ whereas the $z$ vector will only contain $[p_x, p_y]$
  - multiplying $Hx$ allows us to compare $x$, our belief, with $z$, the sensor measurement


- what does the prime notation in the $x$ vector represent?
  - the prime notation like $p_x'$ means you have already done the prediction step but have not done the measurement step yet
  - in other words, the object was at $p_x$ 
  - after time $\Delta{t}$, you calculate where you believe the object will be based on the motion model and get $p_x'$

- now, I would like you to derive the right measurement matrix $H$, which is the projection from a 4D state to a 2D observation state

**Q:** Find the right $H$ matrix to project from a 4D state to a 2D observation space, as follows: $\begin{pmatrix} p_x \\ p_y \end{pmatrix} = H \begin{pmatrix} p_x' \\ p_y' \\ v_x' \\ v_y' \end{pmatrix}$

- Here are your options:
  - A. $H = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$
  - B. $H = \begin{pmatrix} 1 & 0 \\ 0 & 1 \\ 0 & 0 \\ 0 & 0 \end{pmatrix}$
  - C. $H = \begin{pmatrix} 1 & 1 \end{pmatrix}$
  - D. $H = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \end{pmatrix}$

(Hint: first consider the matrix dimensions, then try to use a 0 or 1 to correctly project the components into the measurement space.)


**A:** It's D matrix.
In order to obtain our measurement (x and y) we need to determine the H matrix. (x and y) is found by multiplying H with the state vector.

So let’s find out first what is the dimension of our H matrix. Here we have a 2 by 1 matrix, and that came from our m by n H matrix times the four row and one column matrix. Now, from the matrix multiplication we know that the number of columns of the first matrix should be equal with the number of rows of a second matrix, which is 4. And also the number of rows of the first matrix is the same with the number of rows of the result matrix, which is 2. So our H is a matrix of 2 rows and 4 columns. And finally we put ones and zeroes so that the px and py coordinates are propagated to the result Z.

## Measurement Noise Covariance Matrix R

- now, let's take a look at the covariance matrix, $R$, which represents the uncertainty in our sensor measurements
- the dimensions of the $R$ matrix is square and each side of its matrix is the same length as the number of measurements parameters
- before diving into the details, let's have a quick quiz to test your intuition about what this matrix might look like for the laser sensor

**Q:** What is the dimensionality of the noise covariance matrix, $R$?

**A:** It's 2 by 2 matrix.
Remember that the measurement vector is two dimensional and the two measurements, $p_x$ and $p_y$, have uncorrelated noise components.

- for laser sensors, we have a 2D measurement vector
- each location component $p_x$, $p_y$ are affected by a random noise
- so our noise vector $\omega$ has the same dimension as $z$
  - and it is a distribution with zero mean and a 2 x 2 covariance matrix which comes from the product of the vertical vector $\omega$ and its transpose


- $R = E[\omega \omega^T] = \begin{pmatrix} \sigma^2_{px} & 0 \\ 0 & \sigma^2_{py} \end{pmatrix}$ where R is the measurement noise covariance matrix
  - in other words, the matrix $R$ represents the uncertainty in the position measurements we receive from the laser sensor


- generally, the parameters for the random noise measurement matrix will be provided by the sensor manufacturer
- for the extended Kalman filter project, we have provided $R$ matrices values for both the radar sensor and the lidar sensor
- remember that the off-diagonal $0$s in $R$ indicate that the noise processes are uncorrelated

### Programming assignment

- let's continue with our next programming assignment where you'll actually build a Kalman filter that can take in lidar measurements and track a pedestrian in 2D
- the State Vector of a pedestrian is described by $p_x, p_y$ position and $v_x, v_y$ velocity


- this time, the code is structured into three main parts, a `KalmanFilter` class, a `Tracking` class, and a `MeasurementPackage` class
  - the `KalmanFilter` class is what you've already coded but the whole `KalmanFilter` code including its predict and update functions is now decoupled from the tracking logic
  - the `Tracking` class is where we actually create and use a `KalmanFilter` object
    - for this quiz, you're going to modify $F$ and $Q$ matrices according to the elapsed time between the current and previous measurement
    - you can do so inside the `ProcessMeasurement` function
    - afterwards, your code should call the `KalmanFilter` predict and update functions
    - just keep in mind that all the `KalmanFilter` matrices, as well as the predict update functions are members of the `kf_` object


- there is a big difference between this and previous programming assignment
- here, we don't process the entire list of measurements, but rather the most recent measurement
- we are simulating a more realistic scenario where the process measurement function is called every time we receive new data from a given sensor which, in this case, is only from a laser scanner


- also, the sensor data is organized in package objects now
- these measurement packages are passed as an argument to the process measurement function
  - the package properties are grouped into `MeasurementPackage` class
    - this class is composed of a `timestamp_` variable, which is a point in time where the measurement was created by a sensor and an enum variable called `SensorType` with two options, `LASER` and `RADAR`
      - this is not important right now, as you will only see laser measurement


- there is also vector variable with most recent raw measurements
  - specifically, if `MeasurementPackage` comes from a laser sensor, then the `raw_measurements_` vector will contain just observed $x$ and $y$ position
  - the big number assigned to the timestamp variable is the number of microseconds that have elapsed since Thursday, January 1, 1970 which obviously always increases


- I would like to point out that in this programing assignment the tracking code will first call the predict function and then measurement update function
  - that is because our prediction depends on the elapsed time between two measurements, and the time is not fixed anymore
  - so we can predict our state only after next observation timestamp is available
  - basically we have to wait for the next process measurement call, and then to apply the prediction, right before the next measurement update
  - the first process measurement call will just initialize the state position with the first $x$ and $y$ measurement and the state velocity $v_x$, $v_y$ with $0$, assuming we don't know the initial velocity

### Helpful Equations

- you will be modifying these matrices in the Kalman Filter with the observed time step, `dt`
  - $F = \begin{pmatrix} 1 & 0 & \Delta t & 0 \\ 0 & 1 & 0 & \Delta t \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix}$


- in the tracking class $\sigma_{ax}^2$ = `noise_ax` and $\sigma_{ay}^2$ = `noise_ay`
  - $Q = \begin{pmatrix} \frac{\Delta t^4}{4} \sigma_{ax}^2 & 0 & \frac{\Delta t^3}{2} \sigma_{ax}^2 & 0\\ 0 & \frac{\Delta t^4}{4} \sigma_{ay}^2 & 0 & \frac{\Delta t^3}{2} \sigma_{ay}^2\\ \frac{\Delta t^3}{2}\sigma_{ax}^2& 0 & \Delta t^2 \sigma_{ax}^2 & 0\\ 0 & \frac{\Delta t^3}{2} \sigma_{ay}^2 & 0 & \Delta t^2 \sigma_{ay}^2 \end{pmatrix}$

### Input Data Link

- if you would like to run this coding exercise on your own computer, here is the [input data](https://d17h27t6h515a5.cloudfront.net/topher/2017/March/58bd257a_ekf-data/ekf-data.zip) that is being used

### More Info on Timestamps

- timestamps are often used for logging a sequence of events, so that we know exactly, for example, in our case when the measurements were generated.

- we can use the timestamp values to compute the elapsed time between two consecutive observations as:
```cpp
float delta_t = ( timestamp(k+1) - timestamp(k) ) / 1000000.0
```
- additionally we divide the result by $10^6$ to transform it from microseconds to seconds
```cpp
float dt = (measurement_pack.timestamp_ - previous_timestamp_) / 1000000.0;
```

- code is available in `code/02_laser_measurements_programming_quiz/`

<img src="resources/laser_measurements_vis.png"/>

- here is how the visualization of our results would look
- say, that our pedestrian is moving in a kind of figure eight
  - and as you see here, the orange dots are our laser measurements, which is basically the observed $p_x$ and $p_y$ position
  - the green line is the real path of the pedestrian, or in other words, the ground truth
  - the blue line is the estimated position resulted from our Kalman filter solution


- the estimated trajectory actually might look different depending on how we choose the measurement or process noises
  - for example, if I increase the measurement noise so the measurement covariance matrix, $R$, has two on the diagonal line
  - it works quite well when the pedestrian is moving along the straight line
  - however, our linear motion model is not perfect, especially for the scenarios when the pedestrian is moving along a circular path, as you see here
  - we always predict that the pedestrian position would be somewhere along the straight line, although their real position follows the green trajectory
  - so, our estimation would actually underestimate the truth state.
    - to solve this, instead of using the linear motion model, we can predict the state by using a more complex motion model such as the circular motion
      - you will learn about that in the next lesson


- now that we can update with laser data, let us see what we can do with radar data
- that is the power of the Kalman filter--it can use multiple data sources originating from different sensors to estimate a more accurate object state

# Radar Measurements

- next up, we will integrate radar measurements into our Kalman filter
  - so how will radar data improve our predictions?
- although the laser is providing the pedestrian position with high accuracy, we don't actually have yet a way to observe directly its speed
  - that's where radars can help us
  - using the Doppler effect, the radar can directly measure the radial velocity of a moving object
  - the radial velocity is the component of velocity moving towards or away from the sensor
- however, we also know that the radar has a lower spatial resolution than the laser sensors
  - so why not combine both of these sensors together?


- let's design our Kalman filter to work with new radar information
  - the state transition function will be exactly what you've designed in the LIDAR case: $x' = f(x) + \nu$
  - we use the same state with four parameters: $x = \begin{pmatrix} p_x \\ p_y \\ v_x \\ v_y \end{pmatrix}$ with the same linear motion model and process noise, $z = h(x') + \omega$


- however, our new radar sees the world differently
- let's have our vehicle here and the pedestrian here
- the $x$ axis always points in the vehicle's direction of movement and the $y$ axis points to the left
- instead of a 2D pose, the radar can directly measure:
  - the object range, $\rho$
    - it is a radial distance from origin to our pedestrian
    - we can always define a ray which extends from the origin to our object position
  - bearing, $\varphi$
    - it is the angle between $\rho$ and $x$:
  - range rate (radial velocity - change of $\rho$): $\dot{\rho}$
    - it is the velocity along this ray

<img src="resources/radar_geometry.png"/>

- similar to our motion model, the radar observations are corrupted by a zero-mean random noise, $\omega$ 
- considering that the three measurement vector components are not cross-correlated
  - measurement function: $x' = h(x) + \omega$
  - measurement vector: $z = \begin{pmatrix} \rho \\ \varphi \\ \dot{\rho} \end{pmatrix}$
  - $\omega \sim N(0, R)$
- their radar measurement covariance matrix becomes a three-by-three diagonal matrix $R = \begin{pmatrix} \sigma_{\rho}^2 & 0 & 0 \\ 0 & \sigma_{\varphi}^2 & 0 \\ 0 & 0 & \sigma_{\dot{\rho}}^2 \end{pmatrix}$


- so, our state is still the same and has four parameters, $p_x, p_y, v_x, v_y$ and the measurement vector has three parameters, $\rho, \varphi, \dot{\rho}$
  - $ \begin{pmatrix} \rho \\ \varphi \\ \dot{\rho} \end{pmatrix} \xleftarrow {h(x')} \begin{pmatrix} p_x' \\ p_y' \\ v_x' \\ v_y' \end{pmatrix}$


- the next questing is, what is the measurement function $h(x')$, that maps the predicted state $x'$ into the measurement space?
  - $h$ function basically specifies how the predictive position and speed can be related to the object range, bearing, and range rate
  - looking at the $h$ function, you can see that it is a nonlinear function
  - so comparing to the linear case when using laser data, we don't have a measurement matrix, $H$ here

## H versus h(x)

- the $H$ matrix from the lidar lesson and $h(x)$ equations from the radar lesson are actually accomplishing the same thing; they are both needed to solve $y = z - Hx'$ in the update step
- but for radar, there is no $H$ matrix that will map the state vector $x$ into polar coordinates; instead, you need to calculate the mapping manually to convert from cartesian coordinates to polar coordinates


- here is the $h$ function that specifies how the predicted position and speed get mapped to the polar coordinates of range, bearing and range rate
  - $h(x')= \begin{pmatrix} \rho\\ \varphi\\ \dot{\rho} \end{pmatrix} = \begin{pmatrix} \sqrt{ p{'}_x^2 + p{'}_y^2 }\\ \arctan(p_y' / p_x')\\ \frac{p_x' v_x' + p_y' v_y'}{\sqrt{p{'}_x^2 + p{'}_{y}^2}} \end{pmatrix}$


- hence for radar $y = z - Hx'$ becomes $y = z - h(x')$

## Definition of Radar Variables

- the range, ($\rho$), is the distance to the pedestrian
  - the range is basically the magnitude of the position vector $\rho$ which can be defined as $\rho = \sqrt{p_x^2 + p_y^2}$
- $\varphi = \arctan(p_y / p_x)$
  - note that $\varphi$ is referenced counter-clockwise from the x-axis, so $\varphi$ from the video clip above in that situation would actually be negative
- the range rate, $\dot{\rho}$ is the projection of the velocity, $v$, onto the line, $L$

## Deriving the Radar Measurement Function

- the measurement function is composed of three components that show how the predicted state, $x' = (p_x', p_y', v_x', v_y')^T$ is mapped into the measurement space, $z = (\rho, \varphi, \dot{\rho})^T$:
  - the range, $\rho$, is the distance to the pedestrian which can be defined as: $\rho = \sqrt{p_x^2 + p_y^2}$
  - $\varphi$ is the angle between $\rho$ and the $x$ direction and can be defined as: $\varphi = \arctan(p_y/p_x)$
  - there are two ways to do the range rate $\dot{\rho(t)}$ derivation:
    - generally we can explicitly describe the range, $\rho$, as a function of time: $\rho(t) = \sqrt{p_x(t)^2 + p_y(t)^2}$
    - the range rate, $\dot{\rho(t)}$, is defined as time rate of change of the range, $\rho$, and it can be described as the time derivative of $\dot{\rho} = \frac{\partial \rho(t)}{\partial t} = \frac{ \partial}{\partial t}\sqrt{p_x(t)^2 + p_y(t)^2} = \frac{1}{2 \sqrt{p_x(t)^2 + p_y(t)^2}} (\frac{ \partial}{\partial t}p_x(t)^2 + \frac{ \partial}{\partial t}p_y(t)^2)$    
$=\frac{1}{2 \sqrt{p_x(t)^2 + p_y(t)^2}} (2 p_x(t) \frac{ \partial}{\partial t} p_x(t) + 2 p_y(t) \frac{ \partial}{\partial t} p_y(t))$
      - $\frac{ \partial}{\partial t} p_x(t)$ is nothing else than $v_x(t)$, similarly $\frac{ \partial}{\partial t} p_y(t)$ is $v_y(t)$ so we have:
      - $\dot{\rho} = \frac{\partial \rho(t)}{\partial t} = \frac{1}{2 \sqrt{p_x(t)^2 + p_y(t)^2}} (2 p_x(t) v_x(t) + 2 p_y(t) v_y(t)) = \frac{2( p_x(t) v_x(t) + p_y(t) v_y(t))}{2 \sqrt{p_x(t)^2 + p_y(t)^2}} = \frac{p_x(t) v_x(t) + p_y(t) v_y(t)}{ \sqrt{p_x(t)^2 + p_y(t)^2}}$
    - for simplicity we just use the following notation: $\dot{\rho} = \frac{p_x v_x + p_y v_y}{ \sqrt{p_x^2 + p_y^2}}$
    - the range rate, $\dot{\rho}$, can be seen as a scalar projection of the velocity vector, $\vec{v}$, onto $\vec{\rho}$
    - both $\vec{\rho}$ and $\vec{v}$ are 2D vectors defined as:
      - $\vec{\rho}= \begin{pmatrix} p_x\\ p_y \end{pmatrix}, \space \vec{v}=\begin{pmatrix} v_x\\ v_y \end{pmatrix}$
    - the scalar projection of the velocity vector $\vec{v}$ onto $\vec{\rho}$ is defined as: $\dot{\rho}= \frac{\vec{v} \vec{\rho}}{\lvert \vec{\rho} \rvert} = \frac{ \begin{pmatrix} v_x & v_y \end{pmatrix} \begin{pmatrix} p_x\\ p_y \end{pmatrix} }{ \sqrt{p_x^2 + p_y^2} } = \frac{p_x v_x + p_y v_y}{ \sqrt{p_x^2 + p_y^2}}$ where $\lvert \vec{\rho} \rvert$ is the length of $\vec{\rho}$
      - in our case it is actually the range, so $\rho = \lvert \vec{\rho} \rvert$

# Mapping with a Nonlinear Function

<img src="resources/mapping_witn_nonlinear_function.png"/>

**Q:** What happens if we have a nonlinear measurement function, $h(x)$. Can we apply the Kalman Filter equations to update the predicted state, $X$, with new measurements, $z$?
<br/>
**A:** No! We aren't working with Gaussian distributions after applying a nonlinear measurement function.

# Extended Kalman Filter

- say that we have our predicted state $x$ described by a Gaussian distribution
- if we map this Gaussian to a nonlinear function $h$, then the result is not a Gaussian distribution anymore (it nas multiple peaks)
  - so the Kalman filter is not applicable anymore

<img src="resources/nonlinear_transformation_example_graphs_1.png"/>

- in order to visualize the impact of this nonlinear transformation, I've created a simple test
  - I've generated a list of 10,000 random values drawn from a normal distribution with a mean of $0$ and a standard deviation of $3$ (red histogram of values below)
    - you can see that it follows a Gaussian shape
  - then for each of these values $x_i$, I have computed the corresponding $h$ over $x_i$
    - in my case, $h$ is the arctangent of $x_i$, which is a nonlinear function that is also used in our radar measurement model
    - the result is stored in an output list
  - finally, I use my output list to generate a new histogram (blue histogram below)
    - what we see here is that the resulting distribution is not a Gaussian any more, because of the nonlinearity of $h$
    - moreover, the transformed density cannot be represented in closed form, so Kalman filter update equations are not applicable here
    - a possible solution is to linearize the $(x)$ function


- that's the key idea of the so-called **extended Kalman filter**
  - we have to approximate our measurement function $h$ by a linear function which is tangent to $h$ at the mean location of the original Gaussian

<img src="resources/approximate_measurement_function.png"/>

- so I repeated the same test
  - I used the same list of randomly sampled values from a normal distribution
  - but instead of applying the nonlinear function $h$, all the $x_i$ values were passed through the linear approximation of $h$
  - we can see now that unlike the non linear case, this time, our resulting distribution retained the Gaussian nature

<img src="resources/nonlinear_transformation_example_graphs_2.png"/>

- so how do we linearize a nonlinear function?
- the extended Kalman filter uses a method called first order Taylor expansion
  - what we do is we first evaluate the nonlinear function $h$ at the mean location $\mu$, which is the best estimate of our predicted distribution
  - then we extrapolate the line with slope around $\mu$
    - this slope is given by the first derivative of $h$
  - similarly, extended Kalman filters implement exactly the same linearization when the state transition $f(x)$ of $x$ is also nonlinear


- the orange line represents the first order Taylor expansion of $\arctan(x)$
  - the equation of the line that linearizes the function $h(x)$ at the mean location $\mu$ solution: $h(x) \approx h(\mu) + \frac{\partial h(\mu)}{\partial x}(x-\mu) = \arctan(\mu) + \frac{1}{1+\mu^2}(x-\mu)$
  - in our example $\mu=0$, therefore: $h(x) \approx \arctan(0) + \frac{1}{1+0}(x-0) = x$ so, the function, $h(x) = \arctan(x)$ will be approximated by a line: $h(x) \approx x$

## How to Perform a Taylor Expansion

- the general form of a Taylor series expansion of an equation, $f(x)$, at point $\mu$ is as follows: $f(x) \approx f(\mu) + \frac{\partial f(\mu)}{\partial x} ( x - \mu)f(x)$
- simply replace $f(x)$ with a given equation, find the partial derivative, and plug in the value $\mu$ to find the Taylor expansion at that value of $\mu$
  - see if you can find the Taylor expansion of $arctan(x)$


- let’s say we have a predicted state density described by $\mu = 0$ and $\sigma = 3$
- the function that projects the predicted state, $x$, to the measurement space $z$ is $h(x) = arctan(x)$ and its partial derivative is $\partial h = 1/(1+ x^2)$

# Multivariate Taylor Series Expansion

- now that you’ve seen how to do a Taylor series expansion with a one-dimensional equation, we’ll need to look at the Taylor series expansion for multi-dimensional equations
- recall from the Radar Measurements lecture that the $h$ function is composed of three equations that show how the predicted state, $x' = (p_x', p_y', v_x', v_y')^T$, is mapped into the measurement space, $z = (\rho, \varphi, \dot{\rho})^T$
  - $h(x')= \begin{pmatrix} \sqrt{ p{'}_x^2 + p{'}_y^2 }\\ \arctan(p_y' / p_x')\\ \frac{p_x' v_x' + p_y' v_y'}{\sqrt{p{'}_x^2 + p{'}_{y}^2}} \end{pmatrix}$


- these are multi-dimensional equations, so we will need to use a multi-dimensional Taylor series expansion to make a linear approximation of the $h$ function
- here is a general formula for the multi-dimensional Taylor series expansion: $T(x) = f(a) + (x - a)^T Df(a) + \frac 1{2!}(x-a)^T D^2f(a)(x - a) + ...$ 
  - $Df(a)$ is called the *Jacobian matrix* and $D^2f(a)$ the *Hessian matrix*
  - they represent first order and second order derivatives of multi-dimensional equations
  - a full Taylor series expansion would include higher order terms as well for the third order derivatives, fourth order derivatives, and so on


- notice the similarities between the multi-dimensional Taylor series expansion and the one-dimensional Taylor series expansion: $T(x) = f(a) + \frac{f'(a)}{1!}(x - a) + \frac{f''(a)}{2!}(x-a)^2 + \frac{f'''(a)}{3!}(x - a)^3 + ...$


- to derive a linear approximation for the $h$ function, we will only keep the expansion up to the Jacobian matrix $Df(a)$--we will ignore the Hessian matrix $D^2f(a)$ and other higher order terms
- assuming $(x - a)$ is small, $(x - a)^2$ or the multi-dimensional equivalent $(x-a)^T (x - a)$ will be even smaller; the extended Kalman filter we'll be using assumes that higher order terms beyond the Jacobian are negligible

# Jacobian Matrix

- if we generalize our previous example to a higher dimension, the derivative of $h(x)$, with respect to $x$, is called the **Jacobian** and it's going to be a matrix containing all the partial derivatives

- for example, for my function $h$, in the first row, I have the first dimension of the function $h1$ derived with respect to the first dimension of $x$, second dimension of $x$, to the nth dimension of $x$
- then I repeat this for every dimension of lower case $h$


- to be more specific, I know that the measurement function describes three components: range, bearing, and range rate
- my state is a vector with four components: $p_x, p_y, v_x, v_y$
- in that case, the Jacobian matrix $H_j$ is going to be matrix with three rows and four columns
- after computing all of these partial derivatives, I obtain the matrix

- we're going to calculate, step by step, all the partial derivatives in $H_j$
  - $\Large H_j = \begin{bmatrix} \frac{\partial \rho}{\partial p_x} & \frac{\partial \rho}{\partial p_y} & \frac{\partial \rho}{\partial v_x} & \frac{\partial \rho}{\partial v_y}\\ \frac{\partial \varphi}{\partial p_x} & \frac{\partial \varphi}{\partial p_y} & \frac{\partial \varphi}{\partial v_x} & \frac{\partial \varphi}{\partial v_y}\\ \frac{\partial \dot{\rho}}{\partial p_x} & \frac{\partial \dot{\rho}}{\partial p_y} & \frac{\partial \dot{\rho}}{\partial v_x} & \frac{\partial \dot{\rho}}{\partial v_y} \end{bmatrix}$

- all of $H_j$'s elements are calculated as follows:
  - $\frac{\partial \rho}{\partial p_x}= \frac{\partial}{\partial p_x}(\sqrt{p_x^2 + p_y^2}) = \frac{2 p_x}{2 \sqrt{p_x^2 + p_y^2}} = \frac{p_x}{\sqrt{p_x^2 + p_y^2}}$
  - $\frac{\partial \rho}{\partial p_y}= \frac{\partial}{\partial p_y}(\sqrt{p_x^2 + p_y^2}) = \frac{2 p_y}{2 \sqrt{p_x^2 + p_y^2}} = \frac{p_y}{\sqrt{p_x^2 + p_y^2}}$
  - $\frac{\partial \rho}{\partial v_x} = \frac{\partial}{\partial v_x}(\sqrt{p_x^2 + p_y^2}) = 0$
  - $\frac{\partial \rho}{\partial v_y} = \frac{\partial}{\partial v_y}(\sqrt{p_x^2 + p_y^2}) = 0$
  - $\frac{\partial \varphi}{\partial p_x} = \frac{\partial}{\partial p_x}arctan(p_y/p_x) = \frac{1}{(\frac{p_y}{p_x})^2 + 1} (- \frac{p_y}{p_x^2}) = -\frac{p_y}{p_x^2+p_y^2}$
  - $\frac{\partial \varphi}{\partial p_y} = \frac{\partial}{\partial p_y}arctan(p_y/p_x) = \frac{1}{(\frac{p_y}{p_x})^2 + 1} (\frac{1}{p_x}) = \frac{p_x^2}{p_x^2+p_y^2} \frac{1}{p_x} =\frac{p_x}{p_x^2+p_y^2}$
  - $\frac{\partial \varphi}{\partial v_x} = \frac{\partial}{\partial v_x}arctan(p_y/p_x) = 0$
  - $\frac{\partial \varphi}{\partial v_y} = \frac{\partial}{\partial v_y}arctan(p_y/p_x) = 0$
  - $\frac{\partial \dot{\rho}}{\partial p_x} = \frac{\partial}{\partial p_x} \left( \frac{p_x v_x + p_y v_y}{\sqrt{p_x^2 + p_y^2}} \right)$


- in order to calculate the derivative of this function we use the quotient rule
- given a function $z$ that is quotient of two other functions, $f$ and $g$: $z = \frac{f}{g}$, its derivative with respect to $x$ is defined as: $\frac{\partial z}{\partial x} = \frac { \frac{\partial f}{\partial x} g - \frac{\partial g}{\partial x} f } {g^2}$


- in our case:
  - $f = p_x v_x + p_y v_y$
  - $g = \sqrt{p_x^2 + p_y^2}$
- their derivatives are:
  - $\frac{\partial f}{\partial p_x} = \frac{\partial}{\partial p_x} (p_x v_x + p_y v_y) = v_x$
  - $\frac{\partial g}{\partial p_x} = \frac{\partial}{\partial p_x} \left( \sqrt{p_x^2 + p_y^2} \right) = \frac{p_x}{\sqrt{p_x^2 + p_y^2}}$


- putting everything together into the derivative quotient rule we have: $\frac{\partial \dot{\rho}} {\partial p_x} = \frac{v_x \sqrt{p_x^2 + p_y^2} - \frac{p_x}{\sqrt{p_x^2 + p_y^2}} (p_x v_x + p_y v_y)} {p_x^2 + p_y^2} = \frac{p_y(v_x p_y - v_y p_x)}{(p_x^2 + p_y^2)^{3/2}}$


- similarly:
  - $\frac{\partial \dot{\rho}}{\partial p_y} = \frac{\partial}{\partial p_y} \left( \frac{p_x v_x + p_y v_y}{\sqrt{p_x^2 + p_y^2}} \right) = \frac{p_x(v_y p_x - v_x p_y)}{(p_x^2 + p_y^2)^{3/2}}$
  - $\frac{\partial \dot{\rho}}{\partial v_x} = \frac{\partial}{\partial v_x} \left( \frac{p_x v_x + p_y v_y}{\sqrt{p_x^2 + p_y^2}} \right) = \frac{p_x}{\sqrt{p_x^2 + p_y^2}}$
  - $\frac{\partial \dot{\rho}}{\partial v_y} = \frac{\partial}{\partial v_y} \left( \frac{p_x v_x + p_y v_y}{\sqrt{p_x^2 + p_y^2}} \right) = \frac{p_y}{\sqrt{p_x^2 + p_y^2}}$


- so now, after calculating all the partial derivatives, our resulted Jacobian is:
  - $\Large H_j = \begin{bmatrix} \frac{p_x}{\sqrt{p_x^2 + p_y^2}} & \frac{p_y}{\sqrt{p_x^2 + p_y^2}} & 0 & 0\\ -\frac{p_y}{p_x^2 + p_y^2} & \frac{p_x}{p_x^2 + p_y^2} & 0 & 0\\ \frac{p_y(v_x p_y - v_y p_x)}{(p_x^2 + p_y^2)^{3/2}} & \frac{p_x(v_y p_x - v_x p_y)}{(p_x^2 + p_y^2)^{3/2}} & \frac{p_x}{\sqrt{p_x^2 + p_y^2}} & \frac{p_y}{\sqrt{p_x^2 + p_y^2}}\\ \end{bmatrix}$

- let's program this
- I would like you to implement CalculateJacobian function, which takes our current predicted state and computes the new Jacobian matrix, $H_j$, that you just saw
  - so, for example, if our predicted state  $x$ is equal to $1, 2, 0.2, 0.4$, I would like to output a $3x4$ matrix with the following values
  - you also have to make sure you don't divide by $0$ when computing the Jacobian
    - in other words, you have to check that $x$ and $y$ are not both $0$; if this happens, just print out the error message

- code is available in `/code/03_jacobian_matrix/`

# EKF Algorithm Generalization

<img src="resources/kalman_vs_extended_kalman.jpg"/>

- I can start from my Kalman Filter, previously described
- what I change is I simply use my non-linear function, $f(x)$, to predict the state and $h(x)$ to compute the measurement error
- additionally, the state transition matrix $F$ and the measurement matrix $H$ are replaced by corresponding Jacobians: $F_j$, and $H_j$
  - so first, we have to compute these Jacobian matrices
  - because the linearization points change, I have to recompute the Jacobians at every point in time


- for the Extended Kalman Filter, I first linearize the nonlinear prediction and measurement functions
- then I just use the same mechanism to estimate the new state

## Extended Kalman Filter Equations

- although the mathematical proof is somewhat complex, it turns out that the Kalman filter equations and extended Kalman filter equations are very similar
- the main differences are:
  - the $F$ matrix will be replaced by $F_j$ when calculating $P'$
  - the $H$ matrix in the Kalman filter will be replaced by the Jacobian matrix $H_j$ when calculating $S$, $K$, and $P$
  - to calculate $x'$, the prediction update function, $f$, is used instead of the $F$ matrix
  - to calculate $y$, the $h$ function is used instead of the $H$ matrix


- for this project, however, we do not need to use the $f$ function or $F_j$
- if we had been using a non-linear model in the prediction step, we would need to replace the $F$ matrix with its Jacobian, $F_j$
- however, we are using a linear model for the prediction step
- so, for the prediction step, we can still use the regular Kalman filter equations and the $F$ matrix rather than the extended Kalman filter equations


- the measurement update for lidar will also use the regular Kalman filter equations, since lidar uses linear equations
- only the measurement update for the radar sensor will use the extended Kalman filter equations


- one important point to reiterate is that the equation $y = z - Hx'$ for the Kalman filter does not become $y = z - H_jx$ for the extended Kalman filter
- instead, for extended Kalman filters, we'll use the $h$ function directly to map predicted locations $x'$ from Cartesian to polar coordinates

## Clarification of u = 0

- in the above image, the prediction equation is written as $x' = Fx + u$ and $x' = f(x,u)$
  - previously the equation was written $x' = Fx + \nu$
- it is just a question of notation where $\nu$ is the greek letter "nu" and "u" is used in the code examples
- remember that $\nu$ is represented by a Gaussian distribution with mean zero
- the equation $x' = Fx + u$ or the equivalent equation $x' = Fx + \nu$ calculates the mean value of the state variable $x$; hence we set $u = 0$
- the uncertainty in the gaussian distribution shows up in the $Q$ matrix

## More Details About Calculations with Radar Versus Lidar

- in the radar update step, the Jacobian matrix $H_j$ is used to calculate $S$, $K$ and $P$
- to calculate $y$, we use the equations that map the predicted location $x'$ from Cartesian coordinates to polar coordinates: $h(x')= \begin{pmatrix} \sqrt{ p{'}_{x}^2 + p{'}_{y}^2 }\\ \arctan(p_y' / p_x')\\ \frac{p_x' v_x' + p_y' v_y'}{\sqrt{p{'}_{x}^2 + p{'}_{y}^2}} \end{pmatrix}$	 


- the predicted measurement vector $x'$ is a vector containing values in the form $\begin{bmatrix} p_x, p_y, v_x, v_y \end{bmatrix}$
- the radar sensor will output values in polar coordinates: $\begin{pmatrix} \rho\\ \phi\\ \dot{\rho} \end{pmatrix}$


- in order to calculate $y$ for the radar sensor, we need to convert $x'$ to polar coordinates
  - in other words, the function $h(x)$ maps values from Cartesian coordinates to polar coordinates
  - so the equation for radar becomes $y = z_{radar} - h(x')$


- one other important point when calculating $y$ with radar sensor data: the second value in the polar coordinate vector is the angle $\phi$
  - you'll need to make sure to normalize $\phi$ in the $y$ vector so that its angle is between $-\pi$ and $\pi$
  - in other words, add or subtract $2\pi$ from $\phi$ until it is between $-\pi$ and $\pi$


- to summarize:
  - for measurement updates with lidar, we can use the $H$ matrix for calculating $y$, $S$, $K$ and $P$
  - for radar, $H_j$ is used to calculate $S$, $K$ and $P$

**Q:** Compared to Kalman Filters, how would the Extended Kalman Filter result differ when the prediction function and measurement function are both linear?
<br/>
**A:** The Extended Kalman Filter's result would be the same as the standard Kalman Filter's result.
If $f$ and $h$ are linear functions, then the Extended Kalman Filter generates exactly the same result as the standard Kalman Filter. Actually, if $f$ and $h$ are linear then the Extended Kalman Filter $F_j$ turns into $f$ and $H_j$ turns into $h$. All that's left is the same ol' standard Kalman Filter!
In our case we have a linear motion model, but a nonlinear measurement model when we use radar observations. So, we have to compute the Jacobian only for the measurement function.

# Sensor Fusion General Flow

<img src="resources/sensor_fusion_general_flow.png"/>

- here is my overall processing flow that combines what we've learned so far
- we have a moving pedestrian
  - its state is represented by a 2D position and a 2D velocity in a state vector
- each time we receive new measurements from a given sensor, the estimation function process measurement is triggered
- at first iteration, we just initialize the state and covariance matrix
- subsequently, we call the prediction and measurement update


- before prediction we, have to compute the elapsed time between the current and previous observation
- and based on the elapsed time, we calculate new state transition and process covariance matrices


- the measurement update step depends on sensor type
  - if the current observations come from a radar sensor, then we have to compute the new Jacobian $H_j$, use the non-linear measurement function to project the predicted state and call the measurement update
  - otherwise, if the current observations are generated by a laser sensor, then we just set up the extended Kalman filter with the laser $H$ and $R$ matrices and then call the measurement update

# Evaluating KF Performance

- once we have implemented our tracking algorithm, we might want to check its performance in terms of how far the estimated result is from the true result
- there are many evaluation metrics, but perhaps the most common one is the so-called root mean squared error
  - $RMSE = \sqrt{\frac{1}{n}\sum_{t=1}^{n}{(x_t^{est}-x_t^{true})^2}}$
  - in the context of tracking, it's an accuracy metric used to measure the deviation of the estimated state from the true state


- let's see how it's computed
  - at the given processing step, I basically need two values, the estimated state which is a vector with a position and velocity components, and the real values that are usually known as ground truth values
  - the difference between the estimated state and the ground truth state gives me a so called, residual
  - these residuals are squared and averaged and finally the square root gives me the error metric
    - the lower is the error, the higher is our estimation accuracy

- let's see how it works in C++
- I'm giving you a starting code
- I want you to complete `CalculateRMSE` function, which takes a list of estimated states, and another list of corresponding ground truth states
  - it returns a vector with a root mean square error of the position and velocity
- also consider that you have to check for invalid inputs in the case of the two vectors, estimations and ground truths
  - the estimation vector should be non-empty and it has to be the same size as the ground truth vector
- you will have to use the Eigen functions to perform all the necessary operations

- code is available in `code/04_rmse_kalman/`

# Outro

- at the high level, you've designed the fusion system that can incorporate measurements provided by laser and radar sensors
- thus we can estimate pedestrians' position and speed
- along the way, you learned how to implement a Kalman filter in 2D and you also started working with nonlinear measurement models

# Additional Resources on Sensor Fusion and Object Detection & Tracking

- nice work reaching the end of the sensor fusion content!
- while you still have the project left to do here, we're also providing some additional resources and recent research on the topic that you can come back to if you have time later on


- all of these are completely optional reading - you could spend days reading through the entirety of these!
- we suggest moving onto the project first so you have Kalman Filters fresh on your mind, before coming back to check these out


- we've categorized these papers to hopefully help you narrow down which ones might be of interest, as well as highlighted a couple key reads by category by including their Abstract section, which summarizes the paper
- we've also included some additional papers you might consider as well if you want to delve even deeper

## Tracking Multiple Objects and Sensor Fusion

The below papers and resources concern tracking multiple objects, using Kalman Filters as well as other techniques!

No Blind Spots: Full-Surround Multi-Object Tracking for Autonomous Vehicles using Cameras & LiDARs by A. Rangesh and M. Trivedi https://arxiv.org/pdf/1802.08755.pdf

***Abstract:*** *Online multi-object tracking (MOT) is extremely important for high-level spatial reasoning and path planning for autonomous and highly-automated vehicles. In this paper, we present a modular framework for tracking multiple objects (vehicles), capable of accepting object proposals from different sensor modalities (vision and range) and a variable number of sensors, to produce continuous object tracks. [...] We demonstrate that our framework is well-suited to track objects through entire maneuvers around the ego-vehicle, some of which take more than a few minutes to complete. We also leverage the modularity of our approach by comparing the effects of including/excluding different sensors, changing the total number of sensors, and the quality of object proposals on the final tracking result.*

Multiple Sensor Fusion and Classification for Moving Object Detection and Tracking by R.O. Chavez-Garcia and O. Aycard https://hal.archives-ouvertes.fr/hal-01241846/document

***Abstract:*** *We believe that by including the objects classification from multiple sensors detections as a key component of the object’s representation and the perception process, we can improve the perceived model of the environment. First, we define a composite object representation to include class information in the core object’s description. Second, we propose a complete perception fusion architecture based on the Evidential framework to solve the Detection and Tracking of Moving Objects (DATMO) problem by integrating the composite representation and uncertainty management. Finally, we integrate our fusion approach in a real-time application inside a vehicle demonstrator from the interactIVe IP European project which includes three main sensors: radar, lidar and camera.*


## Stereo cameras

The below papers cover various methods of using stereo camera set-ups for object detection and tracking.

Robust 3-D Motion Tracking from Stereo Images: A Model-less Method by Y.K. Yu, et. al. http://www.cse.cuhk.edu.hk/~khwong/J2008_IEEE_TIM_Stereo%20Kalman%20.pdf

***Abstract:*** *Traditional vision-based 3-D motion estimation algorithms require given or calculated 3-D models while the motion is being tracked. We propose a high-speed extended Kalman filter-based approach that recovers camera position and orientation from stereo image sequences without prior knowledge as well as the procedure for the reconstruction of 3-D structures. [...] The proposed method has been applied to recover the motion from stereo image sequences taken by a robot and a hand-held stereo rig. The results are accurate compared to the ground truths. It is shown in the experiment that our algorithm is not susceptible to outlying point features with the application of a validation gate.*

Vehicle Tracking and Motion Estimation Based on Stereo Vision Sequences by A. Barth (long read) http://hss.ulb.uni-bonn.de/2010/2356/2356.pdf

***Abstract:*** *In this dissertation, a novel approach for estimating trajectories of road vehicles such as cars, vans, or motorbikes, based on stereo image sequences is presented. Moving objects are detected and reliably tracked in real-time from within a moving car. [...] The focus of this contribution is on oncoming traffic, while most existing work in the literature addresses tracking the lead vehicle. The overall approach is generic and scalable to a variety of traffic scenes including inner city, country road, and highway scenarios. [...] The key idea is to derive these parameters from a set of tracked 3D points on the object’s surface, which are registered to a time-consistent object coordinate system, by means of an extended Kalman filter. Combining the rigid 3D point cloud model with the dynamic model of a vehicle is one main contribution of this thesis. [...] The experimental results show the proposed system is able to accurately estimate the object pose and motion parameters in a variety of challenging situations, including night scenes, quick turn maneuvers, and partial occlusions.*

## Deep Learning-based approaches

The below papers include various deep learning-based approaches to 3D object detection and tracking.

Fast and Furious: Real Time End-to-End 3D Detection, Tracking and Motion Forecasting with a Single Convolutional Net by W. Luo, et. al. http://openaccess.thecvf.com/content_cvpr_2018/papers/Luo_Fast_and_Furious_CVPR_2018_paper.pdf

***Abstract:*** *In this paper we propose a novel deep neural network that is able to jointly reason about 3D detection, tracking and motion forecasting given data captured by a 3D sensor. By jointly reasoning about these tasks, our holistic approach is more robust to occlusion as well as sparse data at range. Our approach performs 3D convolutions across space and time over a bird’s eye view representation of the 3D world, which is very efficient in terms of both memory and computation. Our experiments on a new very large scale dataset captured in several north american cities, show that we can outperform the state-of-the-art by a large margin. Importantly, by sharing computation we can perform all tasks in as little as 30 ms.*

VoxelNet: End-to-End Learning for Point Cloud Based 3D Object Detection by Y. Zhou and O. Tuzel https://arxiv.org/abs/1711.06396

***Abstract:*** *Accurate detection of objects in 3D point clouds is a central problem in many applications, such as autonomous navigation, housekeeping robots, and augmented/virtual reality. To interface a highly sparse LiDAR point cloud with a region proposal network (RPN), most existing efforts have focused on hand-crafted feature representations, for example, a bird's eye view projection. In this work, we remove the need of manual feature engineering for 3D point clouds and propose VoxelNet, a generic 3D detection network that unifies feature extraction and bounding box prediction into a single stage, end-to-end trainable deep network. [...] Experiments on the KITTI car detection benchmark show that VoxelNet outperforms the state-of-the-art LiDAR based 3D detection methods by a large margin. Furthermore, our network learns an effective discriminative representation of objects with various geometries, leading to encouraging results in 3D detection of pedestrians and cyclists, based on only LiDAR.*


## Other papers on Tracking Multiple Objects and Sensor Fusion

The below papers and resources concern tracking multiple objects, using Kalman Filters as well as other techniques! We have not included the abstracts here for brevity, but you should check those out first to see which of these you want to take a look at.

Multiple Object Tracking using Kalman Filter and Optical Flow by S. Shantaiya, et. al. http://www.ejaet.com/PDF/2-2/EJAET-2-2-34-39.pdf

Kalman Filter Based Multiple Objects Detection-Tracking Algorithm Robust to Occlusion by J-M Jeong, et. al. https://pdfs.semanticscholar.org/f5a2/bf3df3126d2923a617b977ec2b4e1c829a08.pdf

Tracking Multiple Moving Objects Using Unscented Kalman Filtering Techniques by X. Chen, et. al. https://arxiv.org/pdf/1802.01235.pdf

LIDAR-based 3D Object Perception by M. Himmelsbach, et. al https://velodynelidar.com/lidar/hdlpressroom/pdf/Articles/LIDAR-based%203D%20Object%20Perception.pdf

Fast multiple objects detection and tracking fusing color camera and 3D LIDAR for intelligent vehicles by S. Hwang, et. al. https://www.researchgate.net/publication/309503024_Fast_multiple_objects_detection_and_tracking_fusing_color_camera_and_3D_LIDAR_for_intelligent_vehicles

3D-LIDAR Multi Object Tracking for Autonomous Driving by A.S. Rachman (long read) https://repository.tudelft.nl/islandora/object/uuid%3Af536b829-42ae-41d5-968d-13bbaa4ec736